# PROGRAMA 2

## Objetivo:
- Implementación de una red neuronal para la detección de ciberacoso a partir de una recopilación de tweets (actualmente X's).erior uso.

## Tratamiento y visualización de los datos

In [3]:
#Importación de los datos
import pandas as pd
data_train = pd.read_csv('E:\MATERIAS CURSADAS\ONCEAVO SEMESTRE\PLN\cyberbullying_train.csv')
data_test = pd.read_csv('E:\MATERIAS CURSADAS\ONCEAVO SEMESTRE\\PLN\cyberbullying_test.csv')
data_valid = pd.read_csv('E:\MATERIAS CURSADAS\ONCEAVO SEMESTRE\PLN\cyberbullying_val.csv')

data_train.head()

,tweet_text,cyberbullying_type
0,@AntonSirius @erinspice @prpltnkr @ChiefElk oh...,other_cyberbullying
1,"Ladies ""Bedroom Bully"" The Mix Cd By @GappyRan...",not_cyberbullying
2,RT @_bobbidana: Never thought I'd say this but...,not_cyberbullying
3,She is intellectual terrorists and world suffe...,religion
4,You saudias are not friends of Muslim idiots c...,religion


In [4]:
data_test.head()

,tweet_text,cyberbullying_type
0,"@dankmtl Yeap, as a little Propaganda Nazi for...",religion
1,@LyndseyBoo wow!! Leave lyndsey phone alone!!!...,other_cyberbullying
2,"I'm not a man, only know how to bully people :(",other_cyberbullying
3,@urgedharry @nyazpolitics @greenlinerzjm If yo...,religion
4,Calling people dumb isnt nice :( RT @tayyoung_...,ethnicity


In [5]:
data_valid.head()

,tweet_text,cyberbullying_type
0,But you idiot tagged and more also I'm a Muslim,religion
1,@SEXYLALA That's a dumb nigger never fuck LALA...,ethnicity
2,I never hear hispanics calling each other brow...,ethnicity
3,@Lenny_Banx nigger read it again such a dumb a...,ethnicity
4,She was a mean girl in high school. She and Mi...,age


In [6]:
#Bibliotecas necesarias
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
os.environ["KERAS_BACKEND"] = "torch"
import keras
import torch

torch.__version__, keras.__version__

('2.4.1+cpu', '3.5.0')

- Clasificación en clases de los tipos de twwets que tenemos

In [ ]:
data_train['Label'] = 1